[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/096_ml_unsupervised/096_ml_unsupervised.ipynb)

# Aprendizaje Activo

**LABORATORIO 5 "SIS420" - APRENDIZAJE ACTIVO**

*   Integrantes:

*   Pereira Cuba Claudia
*   Avendaño Cruz Adan
*   Llaves Salas Angeles Joana
*   Barja Coragua Erlinda
*   Mamani Ramos Lizandro


### Aplicacion de aprendizaje activo
En este ejercicio implementaremos el modelo con el Dataset.

Nuestro objetivo es aplicar aprendizaje activo con imagenes de : 256x256 píxeles.
En el siguiente dataset:

Link del Dataset: https://www.kaggle.com/datasets/asaniczka/mammals-image-classification-dataset-45-animals

Link del Repositorio de GitHub Lab-5: https://github.com/clpereirac/SIS420-IA/tree/main/Laboratorios/Lab5

### **Información del Dataset**
Conjunto de datos de clasificación de imágenes de mamíferos (45 animales)

Este conjunto de datos original contiene imágenes de 45 clases diferentes de mamíferos.
### Acerca del conjunto de datos
Las imágenes se encuentran en la estructura ImageNet, cada clase tiene su propia carpeta que contiene las imágenes respectivas. Las imágenes tienen una resolución de 256x256 píxeles.
### Detalles del conjunto de datos:
Para crear este conjunto de datos,

Busqué cada clase de mamífero en Google Imágenes y extraje los enlaces de las imágenes.
Luego descargué las imágenes de tamaño completo de la fuente original y las convertí al formato JPG con una resolución de 256 píxeles.
Durante el proceso, la mayoría de las imágenes fueron reducidas de tamaño y solo unas pocas fueron ampliadas.
Finalmente, revisé manualmente todas las imágenes y eliminé todas las que no encajaban bien en la clasificación de imágenes.

### Posibles ideas de tareas:
Entrene un modelo de clasificación de imágenes utilizando arquitecturas populares como ViT, ResNet o EfficientNet.
Realice aprendizaje por transferencia en este conjunto de datos utilizando modelos previamente entrenados.
Explore diferentes técnicas de aumento de datos para mejorar el rendimiento del modelo.
Ajustar los modelos existentes para mejorar la precisión de la clasificación.
Compare el rendimiento de diferentes modelos en este conjunto de datos.
Utilice el conjunto de datos como referencia para evaluar nuevas técnicas de clasificación de imágenes.

### Convención de nomenclatura de clases:
Todos los archivos tienen nombres en estilo ImageNet.
Kingdom
├── class_1
│   ├── class_1-0001.jpg
│   └── class_1-0002.jpg
├── class_2
│   ├── class_2-0001.jpg
│   └── class_2-0002.jpg
└── class_3
    ├── class_3-0001.jpg
    └── class_3-0002.jpg
Para obtener el nombre de la clase a partir de un nombre de archivo aleatorio, puede hacer .split('-')[0].
No se a dividido el conjunto de datos en train,val,test para que puedas decidir las proporciones de división.



#**Aprendizaje Activo**

El aprendizaje activo (o *Active Learning*) consiste en entrenar modelos de ML de manera iterativa, incluyendo en cada iteración nuevas muestras al dataset focalizando en ejemplos en loa que el modelo tenga más problemas.

In [9]:
# Importaciones necesarias
from google.colab import drive
import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import time

In [10]:
# Montar Google Drive y extraer el ZIP
drive.mount('/content/drive')

# Definir rutas
zip_path = '/content/drive/MyDrive/datasets/archive.zip'
extract_path = '/content/drive/MyDrive/datasets/archive/'

# Extraer el archivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Archivo ZIP extraído exitosamente")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivo ZIP extraído exitosamente


In [12]:
import os
import numpy as np
from PIL import Image

# PASO 3: Cargar y preprocesar las imágenes
dataset_dir = '/content/drive/MyDrive/datasets/archive/mammals/'

# Crear listas para almacenar datos
imagenes_data = []
etiquetas = []

# Obtener clases (carpetas)
clases = os.listdir(dataset_dir)
print(f"Número total de clases: {len(clases)}")

# Cargar imágenes (sin imprimir información de cada clase)
for clase in clases:
    ruta_clase = os.path.join(dataset_dir, clase)

    # Verificar que la carpeta de la clase existe
    if not os.path.exists(ruta_clase):
        continue  # Saltar esta clase si la carpeta no existe

    for archivo in os.listdir(ruta_clase):
        if archivo.endswith('.jpg'):
            try:
                img = Image.open(os.path.join(ruta_clase, archivo))
                img = img.resize((128, 128))  # Redimensionar para manejar memoria
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                img_array = np.array(img) / 255.0  # Normalizar
                img_array = img_array.reshape(-1)  # Aplanar para LogisticRegression
                imagenes_data.append(img_array)
                etiquetas.append(clase)
            except Exception as e:
                pass  # Omitir cualquier error para no interrumpir el procesamiento

# Convertir a arrays de numpy
X = np.array(imagenes_data)
y = np.array(etiquetas)

# Mostrar el resumen final
print(f"Número total de imágenes procesadas: {len(X)}")
print(f"Número total de etiquetas procesadas: {len(y)}")


Número total de clases: 45
Número total de imágenes procesadas: 13751
Número total de etiquetas procesadas: 13751


In [12]:
# Dividir el conjunto de datos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
# Aplanar las imágenes para la regresión logística
X_train_flat = X_train.reshape(X_train.shape[0], -1)  # Aplanar las imágenes de entrenamiento
X_val_flat = X_val.reshape(X_val.shape[0], -1)  # Aplanar las imágenes de validación


In [ ]:
# Entrenar el modelo
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=1000, random_state=42)
log_reg.fit(X_train_flat, y_train)


In [ ]:
# Predecir las probabilidades
probas = log_reg.predict_proba(X_train_flat[:1000])  # Predecir para las primeras 1000 muestras


In [ ]:
# Obtener el índice de la clase con la probabilidad más alta para cada muestra
labels_ixs = np.argmax(probas, axis=1)

# Crear un nuevo array labels que contiene las probabilidades más altas
labels = np.array([proba[ix] for proba, ix in zip(probas, labels_ixs)])

# Seleccionar las 10 muestras con las probabilidades más bajas
sorted_ixs = np.argsort(labels)
k = 10  # Número de imágenes a mostrar

# Obtener las imágenes con las probabilidades más bajas
X_lowest = X_train_flat[sorted_ixs[:k]]  # Asegúrate de que k no exceda la cantidad de imágenes

# Crear la figura para las imágenes
plt.figure(figsize=(15, 8))

# Determinar el número de filas y columnas
n_cols = 5  # Cambia a 5 columnas para mostrar mejor
n_rows = (k // n_cols) + (k % n_cols > 0)

# Iterar sobre las imágenes seleccionadas
for index, img in enumerate(X_lowest[:k]):
    plt.subplot(n_rows, n_cols, index + 1)
    plt.imshow(img.reshape(64, 64, 3))  # Cambia a la forma original
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
log_reg5 = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=4000, random_state=42)
%time log_reg5.fit(X_train_flat[:1000], y_train2)  # Asegúrate de que X_train_flat[:1000] esté disponible

# Evaluar el modelo
score = log_reg5.score(X_val_flat, y_val)
print("Accuracy en el conjunto de validación:", score)
